In [1]:
from PromptOps.std_templates import ShotTemplateFormatter
from PromptOps.test import PromptCompletion, Test
from PromptOps.test_suite import TestSuite

Chommakorn


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\chomm\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\chomm\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\chomm\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
def process_and_save_csv(input_file, output_prefix):
    # Initialize the formatter with the input file
    formatter = ShotTemplateFormatter(input_file)

    # List of perturbation types to apply
    perturbation_types = ['robust', 'taxonomy', 'vocab']

    # Process each perturbation type and save the output
    for perturb_type in perturbation_types:
        print(f"Processing {perturb_type} perturbation...")
        formatted_data = formatter.format_all_rows(shot_type='one', perturb_type=perturb_type)
        output_file = f"{output_prefix}_{perturb_type}_output.csv"
        formatter.save_formatted_data_to_csv(formatted_data, output_file)
        print(f"Saved formatted data with {perturb_type} perturbation to {output_file}")

In [3]:
import pandas as pd
def read_and_display_csv(file_names):
    for file_name in file_names:
        print(f"Reading file: {file_name}")
        # Read the CSV into a DataFrame
        df = pd.read_csv(file_name)
        # Display the DataFrame
        print(df.head())  # Show the first 5 rows
        print("\n")

In [ ]:
# Initialize the PromptCompletion model for Llama
llama_url = "http://127.0.0.1:8000/v1/chat/completions"  # Replace with your Llama model URL
completion = PromptCompletion(
    model_provider="llama",
    model="llama-13b",
    system_content="You are an assistant that answer the question",
    temperature=0.5,
    top_p=0.9,
    max_tokens=150,
    llama_url=llama_url
)

In [ ]:
# completion = PromptCompletion(
#     model_provider="openai",  # Can be "openai", "claude", or "gemini"
#     model="gpt-3.5-turbo",  # Specify the OpenAI model
#     api_key="OpenAI-API-Key"  
# )

In [ ]:
# completion = PromptCompletion(
#     model_provider="claude",
#     model="claude-2.1",  # Make sure to use a valid Claude model name
#     api_key="Claude-API-Key"
# )

# # # Generate a batch of completions
# batch_response = completion.generate_completion(
#     prompt="Tell me a short fun fact about video games!",
#     batch=True  # Enable batch processing
# )
# print(batch_response)

In [ ]:
# # Using Gemini model for a single prompt
# completion = PromptCompletion(
#     model_provider="gemini",
#     model="gemini-1.5-pro",  # Specify the Gemini model
#     api_key="Gemini-API-Key"
# )

# # Generate a completion for a single prompt
# response = completion.generate_completion(prompt="Tell me a short fun fact about space exploration!")
# print(response)


# # Create an instance for Gemini (Google Cloud)
# completion = PromptCompletion(model_provider="gemini", api_key="your_google_cloud_api_key")
# response = completion.generate_completion("What is the capital of France?")
# print(response)

In [5]:
input_csv = "one_shot_example.csv"
output_prefix = "formatted"
process_and_save_csv(input_csv, output_prefix)

Processing robust perturbation...
Saved formatted data with robust perturbation to formatted_robust_output.csv
Processing taxonomy perturbation...
Saved formatted data with taxonomy perturbation to formatted_taxonomy_output.csv
Processing vocab perturbation...
Saved formatted data with vocab perturbation to formatted_vocab_output.csv


In [6]:
csv_files = [
        "formatted_robust_output.csv",
        "formatted_taxonomy_output.csv",
        "formatted_vocab_output.csv"
    ]

read_and_display_csv(csv_files)

Reading file: formatted_robust_output.csv
                                     original_prompt  \
0  Q: Is growing seedless cucumber good for a gar...   
1  Q: Can solar panels generate electricity on cl...   
2  Q: Can regular exercise improve mental health?...   

                                      perturb_prompt perturb_type  \
0  Q: Is growing seedless cucumber good for a gar...       robust   
1  Q: Can solar panels generate electricity on cl...       robust   
2  Q: Can regular exercise improve mental health?...       robust   

  expected_result  
0             Yes  
1             Yes  
2             Yes  


Reading file: formatted_taxonomy_output.csv
                                     original_prompt  \
0  Q: Is growing seedless cucumber good for a gar...   
1  Q: Can solar panels generate electricity on cl...   
2  Q: Can regular exercise improve mental health?...   

                                      perturb_prompt perturb_type  \
0  Q: Is growing seedless cucumber g

In [8]:
# Initialize the test suite
test_suite = TestSuite()

# List of CSV files and their perturbation types
csv_files = [
    ("Robustness", "formatted_robust_output.csv"),
    ("Taxonomy", "formatted_taxonomy_output.csv"),
    ("Vocabulary", "formatted_vocab_output.csv")
]

# Read each CSV and create tests
for perturb_type, file_path in csv_files:
    print(f"Processing file: {file_path} with perturbation: {perturb_type}")
    
    # Load the CSV into a DataFrame
    df = pd.read_csv(file_path)
    
    # Iterate over each row to create a test
    for index, row in df.iterrows():
        test = Test(
            name=f"Test {perturb_type} #{index + 1}",
            prompt=row["original_prompt"],  # Original prompt
            expected_result=row["expected_result"],  # Expected result
            description=f"A test with {perturb_type} perturbation",
            perturb_method=perturb_type,  # Perturbation type
            perturb_text=row["perturb_prompt"],  # Perturbed prompt
        )
        test_suite.add_test(test)


Processing file: formatted_robust_output.csv with perturbation: Robustness
Processing file: formatted_taxonomy_output.csv with perturbation: Taxonomy
Processing file: formatted_vocab_output.csv with perturbation: Vocabulary


In [ ]:
# Run all tests
print("Running all tests...")
test_suite.run_all(completion)

# Summarize the results
results, summary = test_suite.summarize()
print("Summary:", summary)

# Export results
# Move the file result to result folder
test_suite.export_results("test_results.csv", file_format="csv", overwrite=True)
test_suite.export_results("test_results.xlsx", file_format="xlsx", overwrite=True)
test_suite.export_results("test_results.json", file_format="json", overwrite=True)

# The total result failed because we did not provide instruction and context in a certain way through this test.

Running all tests...
Summary: {'total_tests': 9, 'failures': 9, 'fail_rate': 100.0}
